In [413]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, recall_score, accuracy_score
from xgboost import XGBRegressor, XGBClassifier

In [414]:
data = pd.read_csv('aggr.csv')

In [415]:
data

,Unnamed: 0,filename,mean,std,max,median,freq
0,0,пульс 116-90-Обрезка 01.MOV.csv,1.544963,12.867879,2.386911,1.544963,1.0
1,0,пульс 116-90-Обрезка 06.MOV.csv,1.544963,12.867879,2.386911,1.544963,1.0
2,0,пульс 116-90-Обрезка 05.MOV.csv,1.506818,11.138351,2.270500,1.506818,1.0
3,0,пульс 116-90-Обрезка 20.MOV.csv,1.868246,38.540588,3.490343,1.868246,1.0
4,0,пульс 116-90-Обрезка 10.MOV.csv,1.537258,12.501719,2.363161,1.537258,1.0
...,...,...,...,...,...,...,...
69,0,спокойное дыхание по 1 шаблону-Обрезка 03.MOV.csv,1.506818,11.138351,2.270500,1.506818,0.0
70,0,спокойное дыхание по 1 шаблону-Обрезка 06.MOV.csv,1.303431,4.822118,1.698932,1.303431,0.0
71,0,спокойное дыхание по 1 шаблону-Обрезка 05.MOV.csv,1.640498,18.195043,2.691234,1.640498,0.0
72,0,спокойное дыхание по 1 шаблону-Обрезка 02.MOV.csv,1.537258,12.501719,2.363161,1.537258,0.0


In [363]:
for i in data.filename:
    print(i)

пульс 116-90-Обрезка 01.MOV.csv
пульс 116-90-Обрезка 06.MOV.csv
пульс 116-90-Обрезка 05.MOV.csv
пульс 116-90-Обрезка 20.MOV.csv
пульс 116-90-Обрезка 10.MOV.csv
пульс 116-90-Обрезка 15.MOV.csv
пульс 116-90-Обрезка 09.MOV.csv
пульс 116-90-Обрезка 19.MOV.csv
delts3.csv
пульс 116-90-Обрезка 07.MOV.csv
пульс 116-90-Обрезка 17.MOV.csv
пульс 116-90-Обрезка 11.MOV.csv
пульс 116-90-Обрезка 12.MOV.csv
пульс 116-90-Обрезка 04.MOV.csv
пульс 116-90-Обрезка 13.MOV.csv
пульс 116-90-Обрезка 14.MOV.csv
пульс 116-90-Обрезка 18.MOV.csv
пульс 116-90-Обрезка 02.MOV.csv
спокойное дыхание пульс 90-Обрезка 14.MOV.csv
спокойное дыхание пульс 90-Обрезка 15.MOV.csv
спокойное дыхание пульс 90-Обрезка 21.MOV.csv
спокойное дыхание пульс 90-Обрезка 02.MOV.csv
спокойное дыхание пульс 90-Обрезка 12.MOV.csv
спокойное дыхание пульс 90-Обрезка 03.MOV.csv
спокойное дыхание пульс 90-Обрезка 17.MOV.csv
спокойное дыхание пульс 90-Обрезка 22.MOV.csv
спокойное дыхание пульс 90-Обрезка 09.MOV.csv
спокойное дыхание пульс 90-Обре

In [416]:
a = []
for i in data.filename:
    if 'delts3' in i:
        continue
    if 'пульс 116-90' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 9:
            a.append(116)
        elif (int(i[-10]) * 10 + int(i[-9]) > 9) and (int(i[-10]) * 10 + int(i[-9]) <= 11):
            a.append(100)
        elif (int(i[-10]) * 10 + int(i[-9]) >= 12) and (int(i[-10]) * 10 + int(i[-9]) <= 14):
            a.append(93)   
        else:
            a.append(90)  
    elif 'спокойное дыхание пульс 90' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 5:
            a.append(90)
        elif (int(i[-10]) * 10 + int(i[-9]) == 6) :
            a.append(87)
        elif (int(i[-10]) * 10 + int(i[-9]) > 6) and (int(i[-10]) * 10 + int(i[-9]) <= 17):
            a.append(95)   
        else:
            a.append(100)
    elif 'пульс 160-150 4 шаблон' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 16:
            a.append(160)
        else:
            a.append(150)
    elif 'Бутербор17' in i:
        a.append(70)
    elif 'пульс 90-80 видео по 2 шаблону' in i:
        if int(i[-10]) * 10 + int(i[-9]) <= 12:
            a.append(92)
        elif (int(i[-10]) * 10 + int(i[-9]) > 12) and (int(i[-10]) * 10 + int(i[-9]) <= 16):
            a.append(80) 
        else:
            a.append(86)
    else:
        a.append(86)

In [417]:
data = data.loc[data.filename != 'delts3.csv']
data['y'] = a

<ipython-input-417-d67f1591679f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = a


In [418]:
data = data.drop(data.loc[:, data.columns.str.contains('^Unnamed')],axis = 1)
data = data.drop(['filename'],axis = 1)

In [420]:
data = data.drop(['freq'],axis = 1)

In [421]:
X = data.drop(['y'], axis = 1)
y = data['y']

In [380]:
for i in X.freq:
    print(i)


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [369]:
y

0     116
1     116
2     116
3      90
4     100
     ... 
69     86
70     86
71     86
72     86
73     86
Name: y, Length: 72, dtype: int64

In [422]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [439]:
xgb = XGBRegressor()

In [440]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [441]:
y_pred = xgb.predict(X_test)

In [442]:
y_test

27    100
28    100
50     70
23     90
31     90
53     92
7      90
61     80
35     95
73     86
58     80
29     95
32     87
43    160
34     95
57     86
72     86
64     92
44    160
4     100
67     80
52     92
Name: y, dtype: int64

In [443]:
pd.Series(y_pred)

0      94.769501
1      94.769501
2      90.000046
3     121.999207
4      99.000427
5      88.000122
6      89.997772
7     159.997177
8      94.769501
9     104.399460
10    104.399460
11     94.769501
12     94.769501
13     98.750626
14     94.769501
15     89.997772
16    100.000122
17    121.999207
18     98.750626
19    100.000122
20    104.399460
21     94.769501
dtype: float32

In [444]:
abs(y_pred - y_test)

27     5.230499
28     5.230499
50    20.000046
23    31.999207
31     9.000427
53     3.999878
7      0.002228
61    79.997177
35     0.230499
73    18.399460
58    24.399460
29     0.230499
32     7.769501
43    61.249374
34     0.230499
57     3.997772
72    14.000122
64    29.999207
44    61.249374
4      0.000122
67    24.399460
52     2.769501
Name: y, dtype: float64

In [445]:
r2_score(y_test, y_pred)

-0.7791906841192016

In [446]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,mean,std,max,median,y
mean,1.000000,0.906000,0.991143,1.000000,-0.100525
std,0.906000,1.000000,0.953543,0.906000,-0.078229
max,0.991143,0.953543,1.000000,0.991143,-0.097249
median,1.000000,0.906000,0.991143,1.000000,-0.100525
y,-0.100525,-0.078229,-0.097249,-0.100525,1.000000


In [381]:
data


,mean,std,max,median,freq,y
0,1.544963,12.867879,2.386911,1.544963,1.0,116
1,1.544963,12.867879,2.386911,1.544963,1.0,116
2,1.506818,11.138351,2.270500,1.506818,1.0,116
3,1.868246,38.540588,3.490343,1.868246,1.0,90
4,1.537258,12.501719,2.363161,1.537258,1.0,100
...,...,...,...,...,...,...
69,1.506818,11.138351,2.270500,1.506818,0.0,86
70,1.303431,4.822118,1.698932,1.303431,0.0,86
71,1.640498,18.195043,2.691234,1.640498,0.0,86
72,1.537258,12.501719,2.363161,1.537258,0.0,86


In [384]:
df = data.drop(['y'], axis = 1)

In [385]:

df

,mean,std,max,median,freq
0,1.544963,12.867879,2.386911,1.544963,1.0
1,1.544963,12.867879,2.386911,1.544963,1.0
2,1.506818,11.138351,2.270500,1.506818,1.0
3,1.868246,38.540588,3.490343,1.868246,1.0
4,1.537258,12.501719,2.363161,1.537258,1.0
...,...,...,...,...,...
69,1.506818,11.138351,2.270500,1.506818,0.0
70,1.303431,4.822118,1.698932,1.303431,0.0
71,1.640498,18.195043,2.691234,1.640498,0.0
72,1.537258,12.501719,2.363161,1.537258,0.0


In [386]:
X = df.drop(['freq'], axis = 1)

In [387]:
y = df.freq

In [388]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [400]:
xg = XGBClassifier()

In [401]:
xg.fit(X_train, y_train)

[09:26:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\stani\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [402]:
y_pred = xg.predict(X_test)

In [403]:
y_pred

array([0., 0., 1., 0., 1., 0., 0., 2., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 0., 0.])

In [404]:
y_test

27    0.0
28    0.0
50    1.0
23    0.0
31    0.0
53    0.0
7     1.0
61    0.0
35    0.0
73    0.0
58    0.0
29    0.0
32    0.0
43    2.0
34    0.0
57    0.0
72    0.0
64    0.0
44    2.0
4     1.0
67    0.0
52    0.0
Name: freq, dtype: float64

In [412]:
accuracy_score(y_pred, y_test)

0.7272727272727273

In [356]:
y_pre

NameError: name 'y_pre' is not defined